In [76]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import TFIDFRetriever, AzureCognitiveSearchRetriever
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from llama_index.embeddings import LangchainEmbedding
import magic
import os
import nltk

os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://openai-nois-intern.openai.azure.com/'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "search-service01"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] ="public-index-ver1"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9"

# nltk.download('averaged_perceptron_tagger')

# pip install unstructured
# Other dependencies to install https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/unstructured_file.html
# pip install python-magic-bin
# pip install chromadb

In [77]:
# Get your loader ready
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader('D:/PDT-Chatbot/new_backend/data/Data/', glob='**/*.txt', loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)

In [78]:
# Load up your text into documents
documents = loader.load()

In [79]:
# Get your text splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [80]:
# Split your documents into texts
texts = text_splitter.split_documents(documents)

In [81]:
retriever = AzureCognitiveSearchRetriever(content_key='content')

In [82]:
# Load up your LLM
llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    max_tokens=512,
    temperature=1
)

### Sources

In [83]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=retriever,
                                return_source_documents=True)

In [84]:
template = """Given a list documents, answer the question.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
DON'T use other sources than the ones provided.
------
{summaries}
------
QUESTION: {question}
FINAL ANSWER:"""

templ = PromptTemplate(
    input_variables=['summaries', 'question'], 
    output_parser=None, 
    partial_variables={}, 
    template=template, 
    template_format='f-string', 
    validate_template=True
)

In [85]:
qaChain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=templ)

In [113]:
query = "Khách hàng của New Ocean gồm những công ty nào ?"
doc = retriever.get_relevant_documents(query)[:4]

if isinstance(doc, list):
    for x in doc:
        x.metadata['source'] = 'idk'
    
else:
    doc.metadata['source'] = 'idk'
    doc = [doc]

result = qaChain({'input_documents': doc, 'question': query}, return_only_outputs=False)

In [114]:
result['output_text']

' Không biết.'

In [115]:
print(doc)

[Document(page_content='New Ocean rất vui trước sự quan tâm và đóng góp nhiệt tình của các tình nguyện viên dành cho hoạt động, đó thực sự là những hành động đẹp, đầy ý nghĩa nhân văn dành cho cuộc sống này. Trân trọng cảm ơn tấm lòng, giọt máu quý báu của các anh chị, Quý khách hàng, đối tác Công ty New Ocean và các tình nguyện viên đã nhiệt tình tham gia chương trình. Rất mong tiếp tục nhận được sự quan tâm, đồng hành của các anh, chị trong các chương trình tình nguyện tiếp theo của công ty New Ocean.\r\nSự kiện\r\nNEW OCEAN THAM DỰ TRIỂN LÃM VIMF 2023\r\nGIẢI PHÁP KIỂM TRA CHẤT LƯỢNG IN ẤN TRONG NGÀNH DƯỢC PHẨM\r\nNew Ocean Automation System\r\nROBOT CỘNG TÁC XẾP HÀNG LÊN PALLET TRONG NGÀNH CÔNG NGHIỆP SẢN XUẤT\n', metadata={'@search.score': 16.880205, 'metadata_storage_path': 'aHR0cHM6Ly9hY3NjaGF0Ym90bm9pc2ludGVybi5ibG9iLmNvcmUud2luZG93cy5uZXQvcHVibGljLWRhdGEtdmVyMS8xNDgudHh00', 'source': 'idk'}), Document(page_content='Với mục đích cung cấp thông tin, mang đến cho khách hàng những